In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crab-age-prediction/CrabAgePrediction.csv
/kaggle/input/playground-series-s3e16/sample_submission.csv
/kaggle/input/playground-series-s3e16/train.csv
/kaggle/input/playground-series-s3e16/test.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
from copy import deepcopy
from functools import partial
from itertools import combinations
import random
import gc
import time

# Import sklearn classes for model selection, cross validation, and performance evaluation
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
from category_encoders import OrdinalEncoder, CountEncoder, CatBoostEncoder, OneHotEncoder
from sklearn.preprocessing import FunctionTransformer, LabelEncoder # OneHotEncoder
from sklearn.compose import ColumnTransformer
from imblearn.under_sampling import RandomUnderSampler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.decomposition import PCA, NMF

# Import libraries for Hypertuning
import optuna

import xgboost as xgb
import lightgbm as lgb
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, LassoCV
from sklearn.linear_model import PassiveAggressiveRegressor, ARDRegression, RidgeCV, ElasticNetCV
from sklearn.linear_model import TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.cross_decomposition import PLSRegression
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from catboost import Pool

!pip install sklego
from sklego.linear_model import LADRegression # Least Absolute Deviation Regression

from sklearn.feature_selection import RFE, RFECV
from sklearn.inspection import permutation_importance

/opt/conda/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.8 MB/s eta 0:00:00


In [3]:
filepath = '/kaggle/input/playground-series-s3e16'

df_train = pd.read_csv(os.path.join(filepath, 'train.csv'))
df_test = pd.read_csv(os.path.join(filepath, 'test.csv'))
original = pd.read_csv('/kaggle/input/crab-age-prediction/CrabAgePrediction.csv')

In [4]:
submission = pd.read_csv("/kaggle/input/playground-series-s3e16/sample_submission.csv")
submission.head()

,id,Age
0,74051,10
1,74052,10
2,74053,10
3,74054,10
4,74055,10


In [5]:
df_train['is_generated'] = 1
df_test['is_generated'] = 1
original['is_generated'] = 0

In [6]:
train = pd.concat([df_train.drop('id',axis=1), original], ignore_index=True)
test = df_test
data = pd.concat([train, test.drop('id',axis=1)], ignore_index=True)

In [7]:
test.drop("id", axis = 1, inplace = True)

In [8]:
le = LabelEncoder()
train["Sex"] = le.fit_transform(train["Sex"])
test["Sex"] = le.transform(test["Sex"])
data["Sex"] = le.transform(data["Sex"])

In [9]:
h1 = data[data['Height'] != 0]
h2 = data[data["Height"] == 0]
print(h1.shape, h2.shape)

(127272, 10) (40, 10)


In [10]:
x_h1 = h1.drop(columns=["Height", "Age", "is_generated"], axis=1)
y_h1 = h1["Height"]
x_h2 = h2.drop(columns=["Height", "Age", "is_generated"], axis=1)

# Add correlated features to the training and prediction data
correlated_features = ["Length", "Diameter", "Shell Weight"]
x_h1 = x_h1[correlated_features]
x_h2 = x_h2[correlated_features]

rfr = RandomForestRegressor(n_jobs=-1, random_state=28)
rfr.fit(x_h1, y_h1)
preds_height = rfr.predict(x_h2)

cnt = 0
for i in range(len(data)):
    if data.iloc[i, data.columns.get_loc("Height")] == 0:
        data.iloc[i, data.columns.get_loc("Height")] = preds_height[cnt]
        cnt += 1

data["Height"].describe()



count    127312.000000
mean          0.348414
std           0.092401
min           0.012500
25%           0.300000
50%           0.362500
75%           0.412500
max           2.825000
Name: Height, dtype: float64

In [11]:
test["Viscera Ratio"] = test["Viscera Weight"] / test["Weight"]
test["Shell Ratio"] = test["Shell Weight"] / test["Weight"]
test["Surface Area"] = 2 * (test["Length"] * test["Diameter"] + test["Length"] * test["Height"] + test["Diameter"] * test["Height"])
test["Volume"] = test["Length"] * test["Diameter"] * test["Height"]
test["Density"] = test["Weight"] / test["Volume"]
test['Shell-to-Body Ratio'] = test['Shell Weight'] / (test['Weight'] + test['Shell Weight'])
test['Meat Yield'] = test['Shucked Weight'] / (test['Weight'] + test['Shell Weight'])
test['Body Condition Index'] = np.sqrt(test['Length'] * test['Weight'] * test['Shucked Weight'])
test['Pseudo BMI'] = test['Weight'] / (test['Height'] ** 2)
test['Len-to-Diam'] = test['Length'] / test['Diameter']
test['wieght-to-Viswieght'] = test['Weight'] / test['Viscera Weight']
test['wieght-to-Shellwieght'] = test['Weight'] / test['Shell Weight']
test['wieght-to-Shckwieght'] = test['Weight'] / test['Shucked Weight']
test["Weight_wo_Viscera"] = test['Shucked Weight'] - test['Viscera Weight']
test['Length^2'] = test['Length'] ** 2
test['Diameter^2'] = test['Diameter'] ** 2
test['Log Weight'] = np.log(test['Weight'] + 1)
test['Length Bins'] = pd.cut(test['Length'], bins=4, labels=False)



In [12]:
train["Viscera Ratio"] = train["Viscera Weight"] / train["Weight"]
train["Shell Ratio"] = train["Shell Weight"] / train["Weight"]
train["Surface Area"] = 2 * (train["Length"] * train["Diameter"] + train["Length"] * train["Height"] + train["Diameter"] * train["Height"])
train["Volume"] = train["Length"] * train["Diameter"] * train["Height"]
train["Density"] = train["Weight"] / train["Volume"]
train['Shell-to-Body Ratio'] = train['Shell Weight'] / (train['Weight'] + train['Shell Weight'])
train['Meat Yield'] = train['Shucked Weight'] / (train['Weight'] + train['Shell Weight'])
train['Body Condition Index'] = np.sqrt(train['Length'] * train['Weight'] * train['Shucked Weight'])
train['Pseudo BMI'] = train['Weight'] / (train['Height'] ** 2)
train['Len-to-Diam'] = train['Length'] / train['Diameter']
train['wieght-to-Viswieght'] = train['Weight'] / train['Viscera Weight']
train['wieght-to-Shellwieght'] = train['Weight'] / train['Shell Weight']
train['wieght-to-Shckwieght'] = train['Weight'] / train['Shucked Weight']
train["Weight_wo_Viscera"] = train['Shucked Weight'] - train['Viscera Weight']
train['Length^2'] = train['Length'] ** 2
train['Diameter^2'] = train['Diameter'] ** 2
train['Log Weight'] = np.log(train['Weight'] + 1)
train['Length Bins'] = pd.cut(train['Length'], bins=4, labels=False)



In [13]:
X_train = train.drop(columns=["Age", "is_generated"], axis=1)
y_train = train.Age
X_test = test.drop("is_generated", axis=1)

In [14]:
def LightGBM(X, y, test_data, params):
    kf = list(KFold(n_splits=10, shuffle=True, random_state=100).split(X, y))
    preds, models = [], []
    oof = np.zeros(len(X))
    imp = pd.DataFrame()
    
    for nfold in np.arange(10):
        print("-"*30, "fold:", nfold, "-"*30)
        
        # set train/valid data
        idx_tr, idx_va = kf[nfold][0], kf[nfold][1]
        x_tr, y_tr = X.loc[idx_tr, :], y.loc[idx_tr]
        x_va, y_va = X.loc[idx_va, :], y.loc[idx_va]
        
        # training
        model = lgb.LGBMRegressor(**params)
        model.fit(x_tr, y_tr,
                eval_set=[(x_tr, y_tr), (x_va, y_va)],
                early_stopping_rounds=300,
                verbose=500,
        )
        models.append(model)
        
        # validation
        pred_va = model.predict(x_va)
        oof[idx_va] = pred_va
        print("MAE(valid)", nfold, ":", "{:.4f}".format(mean_absolute_error(y_va, pred_va)))
        
        # prediction
        pred_test = model.predict(test_data)
        preds.append(pred_test)
        
        # importance
        _imp = pd.DataFrame({"features": X.columns, "importance": model.feature_importances_, "nfold": nfold})
        imp = pd.concat([imp, _imp], axis=0, ignore_index=True)
    
    imp = imp.groupby("features")["importance"].agg(["mean", "std"])
    imp.columns = ["importance", "importance_std"]
    imp["importance_cov"] = imp["importance_std"] / imp["importance"]
    imp = imp.reset_index(drop=False)
    display(imp.sort_values("importance", ascending=False, ignore_index=True))
    
    return preds, models, oof, imp

In [15]:
X_test

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Viscera Ratio,Shell Ratio,...,Pseudo BMI,Len-to-Diam,wieght-to-Viswieght,wieght-to-Shellwieght,wieght-to-Shckwieght,Weight_wo_Viscera,Length^2,Diameter^2,Log Weight,Length Bins
0,1,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552,0.200658,0.315789,...,113.960304,1.377049,4.983607,3.166667,2.356589,1.927766,1.102500,0.581406,2.263662,1
1,1,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930,0.209324,0.255941,...,205.053574,1.309859,4.777293,3.907143,2.205645,3.784658,1.351406,0.787656,2.803795,2
2,0,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415,0.266537,0.330739,...,137.956383,1.303797,3.751825,3.023529,2.622449,1.672620,1.657656,0.975156,2.745452,2
3,0,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676,0.230769,0.247752,...,188.988800,1.569620,4.333333,4.036290,2.120763,6.832230,2.402500,0.975156,3.380241,2
4,1,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066,0.209639,0.283133,...,170.739846,1.308824,4.770115,3.531915,2.128205,3.061746,1.237656,0.722500,2.546710,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49363,0,1.3000,1.0375,0.3250,16.315137,6.690482,5.173784,3.756309,0.317116,0.230235,...,154.462838,1.253012,3.153425,4.343396,2.438559,1.516698,1.690000,1.076406,2.851581,2
49364,1,1.0375,0.7625,0.2625,10.276694,4.436697,1.998640,3.543687,0.194483,0.344828,...,149.140227,1.360656,5.141844,2.900000,2.316294,2.438057,1.076406,0.581406,2.422738,1
49365,0,1.4875,1.1625,0.3625,31.382897,11.396499,6.846404,8.788345,0.218157,0.280036,...,238.823469,1.279570,4.583851,3.570968,2.753731,4.550095,2.212656,1.351406,3.477630,2
49366,0,1.2375,0.9500,0.2875,15.663099,6.095142,3.727959,4.961163,0.238009,0.316742,...,189.496847,1.302632,4.201521,3.157143,2.569767,2.367183,1.531406,0.902500,2.813197,2


In [16]:
X_train

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Viscera Ratio,Shell Ratio,...,Pseudo BMI,Len-to-Diam,wieght-to-Viswieght,wieght-to-Shellwieght,wieght-to-Shckwieght,Weight_wo_Viscera,Length^2,Diameter^2,Log Weight,Length Bins
0,1,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,0.229452,0.288160,...,206.031566,1.297872,4.358209,3.470289,2.276169,6.080968,2.325625,1.380625,3.400303,2
1,1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,0.223129,0.326531,...,137.764512,1.333333,4.481707,3.062500,2.304075,2.197086,1.210000,0.680625,2.435230,1
2,2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,0.224256,0.268879,...,176.195292,1.247191,4.459184,3.719149,2.185000,5.783298,1.925156,1.237656,3.249501,2
3,0,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,0.216970,0.296027,...,202.642226,1.203540,4.608925,3.378072,2.488858,9.363103,2.890000,1.995156,3.944695,3
4,1,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,0.193548,0.255630,...,204.458617,1.234568,5.166667,3.911905,1.944379,7.470093,1.562500,1.025156,3.190028,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77939,0,1.4625,1.1375,0.3250,24.819987,11.651644,5.854172,6.378637,0.235865,0.256996,...,234.982128,1.285714,4.239709,3.891111,2.130170,5.797473,2.138906,1.293906,3.251149,2
77940,0,1.5500,1.2125,0.4375,34.458817,15.450477,7.172423,9.780577,0.208145,0.283834,...,180.029739,1.278351,4.804348,3.523188,2.230275,8.278054,2.402500,1.470156,3.568372,2
77941,1,0.6250,0.4625,0.1625,2.012815,0.765436,0.524466,0.637864,0.260563,0.316901,...,76.224928,1.351351,3.837838,3.155556,2.629630,0.240971,0.390625,0.213906,1.102875,0
77942,1,1.0625,0.7750,0.2625,10.347568,4.507570,2.338834,2.976698,0.226027,0.287671,...,150.168780,1.370968,4.424242,3.476190,2.295597,2.168737,1.128906,0.600625,2.429003,1


In [17]:
lgb_params = {
    "objective": "regression_l1",  
    "metric": "mae",  
    "learning_rate": 0.03,  
    "n_estimators": 10000,  
    "max_depth": 8,  
    "num_leaves": 255,  
    "feature_fraction": 0.4,  
    "min_data_in_leaf": 256,  
    "subsample": 0.4,  
    "reg_alpha": 0.1,  
    "reg_lambda": 0.1,  
    
}

In [18]:
# Training
preds_lgb_fe, models_lgb_fe, oof_lgb_fe, imp_lgb_fe = LightGBM(X_train, y_train, X_test, lgb_params)

# MAE for LightGBM
oof_lgb = np.zeros(len(y_train))

oof_lgb_round = np.zeros(len(oof_lgb), dtype=int)
for i in range(len(oof_lgb)):
    oof_lgb_round[i] = int((oof_lgb_fe[i] * 2 + 1) // 2)

print("MAE(int):", "{:.4f}".format(mean_absolute_error(y_train, oof_lgb_round)))
print("MAE(float):", "{:.4f}".format(mean_absolute_error(y_train, oof_lgb_fe)))

# visualization of predictions by test-data
mean_preds_lgb = np.mean(preds_lgb_fe, axis=0)
mean_preds_lgb_round = np.zeros(len(mean_preds_lgb), dtype=int)
for i in range(len(mean_preds_lgb_round)):
    mean_preds_lgb_round[i] = int((mean_preds_lgb[i] * 2 + 1) // 2)

------------------------------ fold: 0 ------------------------------
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=256, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=256


/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[500]	training's l1: 1.30568	valid_1's l1: 1.35636
[1000]	training's l1: 1.29011	valid_1's l1: 1.35498
MAE(valid) 0 : 1.3548
------------------------------ fold: 1 ------------------------------
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=256, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=256


/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[500]	training's l1: 1.29898	valid_1's l1: 1.39339
[1000]	training's l1: 1.28104	valid_1's l1: 1.39234
[1500]	training's l1: 1.26779	valid_1's l1: 1.3917
MAE(valid) 1 : 1.3916
------------------------------ fold: 2 ------------------------------
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=256, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=256


...

[500]	training's l1: 1.30469	valid_1's l1: 1.34349
[1000]	training's l1: 1.28885	valid_1's l1: 1.34136
[1500]	training's l1: 1.27707	valid_1's l1: 1.3409
MAE(valid) 8 : 1.3408
------------------------------ fold: 9 ------------------------------
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=256, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=256


/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[500]	training's l1: 1.30752	valid_1's l1: 1.34696
[1000]	training's l1: 1.2928	valid_1's l1: 1.34634
MAE(valid) 9 : 1.3462


,features,importance,importance_std,importance_cov
0,Shell Weight,3181.5,465.553971,0.146332
1,Meat Yield,2972.2,578.386607,0.194599
2,Shell Ratio,2555.9,486.334007,0.190279
3,wieght-to-Shckwieght,2547.2,499.979733,0.196286
4,Density,2464.1,593.417681,0.240825
5,Pseudo BMI,2314.6,534.297711,0.230838
6,Len-to-Diam,2313.2,559.660373,0.241942
7,Shucked Weight,2295.3,425.399172,0.185335
8,Viscera Ratio,2293.4,438.984232,0.191412
9,Weight,2140.8,331.662077,0.154924


MAE(int): 1.3387
MAE(float): 1.3568


In [19]:
submission["Age"] = np.round(mean_preds_lgb_round)

# Save submission dataframe to CSV file
submission.to_csv('submission_lgb.csv', index=False)

# Read the submission file
submission_lgb = pd.read_csv('submission_lgb.csv')


print(submission_lgb)

           id  Age
0       74051    7
1       74052    8
2       74053   10
3       74054    9
4       74055    7
...       ...  ...
49363  123414    9
49364  123415    8
49365  123416   13
49366  123417   10
49367  123418   11

[49368 rows x 2 columns]
